In [ ]:
#convolurions only helps to extract the features but they do not classify
#to classufy images in CNN, we simply put Fully Connected Layer at the end (perceptron)
#Pooling layer - not necessary to use. it reduce the siz of image.
#Fully connected layer - we dont flatten the image instead we flatten the features that we got at the end of last cnn layer and pass it to mlp
#training adjust weight and values inside the convalutions
#at each levels convalutions are applied parallel but at each level, convolutions are applied subsequntly (result of 1st layer goes to 2nd etc)
# stride - how much we want to move our filer(by default it is 1)

In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.image as img
from torch import nn   #stands for neural networks
import matplotlib.pyplot as plt
import torchmetrics

In [ ]:
!pip install torchmetrics

In [6]:
training_data = datasets.FashionMNIST(root="data",train=True,download=True, transform=ToTensor())
test_data = datasets.FashionMNIST(root="data",train=False,download=True, transform=ToTensor())

100%|██████████| 26421880/26421880 [00:08<00:00, 3233415.58it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 198540.69it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3697745.53it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 15184442.33it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [7]:
print(training_data)

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()


In [8]:
# create the model
device = ("cuda" if torch.cuda.is_available() else "cpu")

**Sequencial** part helps to execute things sequencially such that output of the firt layer goes to the input of the second layer. Inside Sequential, we must define all the layers.
  When weights togther with the data reaches the threshold, neuron fires,
  we set this threshold with **activation funtion**. (Sigmoid is an example)

Then, we do the **forward pass** which defines how data goes from input later to output layer.



In [9]:
class OurCNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.cnn = nn.Sequential(


      nn.Conv2d(3,5,3),   #1st layer is input layer, 1 gray scale image has only one channel, 5 is number of filters, size of kernel # (64,1,28,28)
      #signoid can be used but it is quite old throug
      nn.ReLU(), #we can use relu (rectified linear unit), activation function. it computer max value between 0 and layer above
      nn.Conv2d(5,10,3), #(64,5,26,26) d-k+1 (dimention - kernel size)
      nn.ReLU(),
      nn.MaxPool2d(2)
    )


    self.mlp = nn.Sequential(
      #we want use extracted features to classify. as we said we use perceptron

      nn.Linear(12*12*10,10), #we can put any number instead 2 bc result of flatten layer will be put as input
      nn.ReLU(),
      nn.Linear(10,10) #1st 10 - output size of above level, 2nd 10 number of classes

      )



       #layer

  def forward(self,x):

      x = self.cnn(x)
      x = torch.flatten(x,1) #reducing the size to 1 to do the classification
      x = self.mlp(x)
      return x
# shape of the image
# (B,C,W,H)

#input size in mlp should be equal to the number of pixels of the image, while cnn does not care about that
#FCN uses CNN even in last layer instead of MLP. we use it for segmentation

In [10]:
# HERE WE WANT TO IMPLEMENT THE MODEL WITHOUT SEQUENTIAL
class OurCNN(nn.Module):
  def __init__(self):
    super().__init__()


    self.input_layer = nn.Conv2d(1,5,3)

    self.relu = nn.ReLU()
    self.conv_1 = nn.Conv2d(5,10,3)

    self.max_pool = nn.MaxPool2d(2) #take max from 2 by 2 kernel


    self.input_linear = nn.Linear(12*12*10,10)

    self.output_linear = nn.Linear(10,10)


  def forward(self,x):
      x = self.input_layer(x)
      x = self.relu(x)
      x = self.conv_1(x)
      x = self.relu(x)
      x = self.max_pool(x)


      x = torch.flatten(x,1)

      x = self.input_linear(x)
      x = self.relu(x)
      x = self.output_linear(x)

      return x


In [11]:
model = OurCNN().to(device)

In [12]:
# define hyperparameters
epochs = 2 # how many times our model analyse the data.
batch_size = 64
learning_rate = 0.001 # amount of the weight change in each iteration. lower rate means, better training however it takes more time

In [13]:
#create the dataloader
train_dataloader = DataLoader(training_data,batch_size=batch_size)
test_dataloader = DataLoader(test_data,batch_size=batch_size)

In [14]:
# define the loss function
loss_fn = nn.CrossEntropyLoss() # we may use any loss functions. it takes arrays of logits and turns into array of prob.
#to compute the error
#define optimizer
optimizer = torch.optim.AdamW(model.parameters(),lr=learning_rate)
# we may change SGD to AdamW

In [15]:
# create the accucary metric
metric = torchmetrics.Accuracy(task="multiclass",num_classes=10)


In [16]:
#define the training loop
def train_loop(dataloader, model,loss_fn,optimizer):

  #get the batch from the dataset
  for batch,(X,y) in enumerate(dataloader):
    #compute prediction and loss
    pred = model(X)
    #y_tensor = torch.tensor([y])
    loss = loss_fn(pred,y)  #since we are passing 64 elements of tensor, convertion is not necessary. earlier we were passing only 1 data at a time

    #backpropogation
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    #print the loss during the training, we do not need to use the below code to print there exists a package to do it
    if batch % 100 == 0:
      acc = metric(pred,y)
      print(f"accuracy current batch: {acc}")

  acc = metric.compute()
  print(f"training final accuracy for each epoch : {acc}")
  metric.reset() #reset the accuracy




In [17]:
#test loop
def test_loop(dataloader, model, loss_fn):
  #we dont use optimizer bc it is already optimized in training stage, optimizer serves to change the weight
  size = len(dataloader)
  num_batches = len(dataloader)
  test_loss, correct = 0,0
  #disable weight update bc we are testing

  with torch.no_grad():
    for X,y in dataloader:
      pred = model(X)
      acc = metric(pred,y)
  acc = metric.compute()
  print(f"testing final accuracy for each epoch : {acc}")
  metric.reset() #reset the accuracy


In [18]:
#train the model
for t in range(epochs):
  print(f"epochs: {t}")
  train_loop(train_dataloader,model, loss_fn,optimizer)
  test_loop(test_dataloader,model,loss_fn)


epochs: 0
accuracy current batch: 0.046875
accuracy current batch: 0.390625
accuracy current batch: 0.8125
accuracy current batch: 0.65625
accuracy current batch: 0.65625
accuracy current batch: 0.75
accuracy current batch: 0.734375
accuracy current batch: 0.765625
accuracy current batch: 0.671875
accuracy current batch: 0.796875
training final accuracy for each epoch : 0.628125011920929
testing final accuracy for each epoch : 0.7734000086784363
epochs: 1
accuracy current batch: 0.828125
accuracy current batch: 0.765625
accuracy current batch: 0.859375
accuracy current batch: 0.796875
accuracy current batch: 0.71875
accuracy current batch: 0.828125
accuracy current batch: 0.828125
accuracy current batch: 0.8125
accuracy current batch: 0.765625
accuracy current batch: 0.828125
training final accuracy for each epoch : 0.8031250238418579
testing final accuracy for each epoch : 0.800599992275238


In [ ]:
# since perceptron requires flattening the dimnetion to 1. and we loose information. it makes mlp not well for images

In [ ]:
#to optimize, we must play with the hyperparameters manually